# Translation
For convenience, we'll use a dummy fetcher used for [testing](https://github.com/rsundqvist/rics/blob/master/src/rics/translation/testing.py) to generates translation data instead of fetching real translations. Generated data is on the form `<placeholder-name>-of-<id>`, eg `'placeholder0-of-id0'`. Since we use the `'{x}, {y}'` translation format, all translated values will take form form `'x-of-<id>, y-of-<id>'`.

In [1]:
from rics.translation import Translator

NAMES_TO_TRANSLATE = "name"
translate = Translator(fmt="{x}, {y}").translate

/home/dev/git/rics/src/rics/translation/_translator.py:156: UserWarning: No fetcher given. Translation data will be automatically generated.
  warnings.warn("No fetcher given. Translation data will be automatically generated.", UserWarning, stacklevel=-1)


## Translating `dict` keys

When translating using `inplace=False` (the default), the `Translator` will always try to return an object of the same type. 
The `Translator` doesn't know what to do with the `dict_keys` class, so we wrap the values we want to translate using a known type.

In [2]:
a_dict = {f"k{i}": i for i in range(4)}
print(a_dict)

translated_keys = translate(list(a_dict), names=NAMES_TO_TRANSLATE)
{tk: a_dict[k] for k, tk in zip(a_dict, translated_keys)}  # tk is translated

{'k0': 0, 'k1': 1, 'k2': 2, 'k3': 3}


{'x-of-k0, y-of-k0': 0,
 'x-of-k1, y-of-k1': 1,
 'x-of-k2, y-of-k2': 2,
 'x-of-k3, y-of-k3': 3}

## Translating Pandas `index` and `columns`
Recipies for translating involving `pandas` types. 

The `attribute` argument may be used to translate a member variable instead.

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(
    data=[["00", "01", "02", "03"], ["10", "11", "12", "13"], ["20", "21", "22", "23"]],
    columns=[f"col{i}" for i in range(4)],
    index=[f"idx{i}" for i in range(3)],
)
df.index.name = NAMES_TO_TRANSLATE
s = df.col0
df

,col0,col1,col2,col3
name,,,,
idx0,00,01,02,03
idx1,10,11,12,13
idx2,20,21,22,23


In [5]:
translate(df, inplace=True)
translate(df, attribute="columns")  # Returns df after overwriting df.columns
translate(df, attribute="index")  # Returns df after overwriting df.index

,"x-of-col0, y-of-col0","x-of-col1, y-of-col1","x-of-col2, y-of-col2","x-of-col3, y-of-col3"
name,,,,
"x-of-idx0, y-of-idx0","x-of-00, y-of-00","x-of-01, y-of-01","x-of-02, y-of-02","x-of-03, y-of-03"
"x-of-idx1, y-of-idx1","x-of-10, y-of-10","x-of-11, y-of-11","x-of-12, y-of-12","x-of-13, y-of-13"
"x-of-idx2, y-of-idx2","x-of-20, y-of-20","x-of-21, y-of-21","x-of-22, y-of-22","x-of-23, y-of-23"


In [6]:
translate(s, inplace=True)
translate(s, attribute="index")

name
x-of-idx0, y-of-idx0    x-of-00, y-of-00
x-of-idx1, y-of-idx1    x-of-10, y-of-10
x-of-idx2, y-of-idx2    x-of-20, y-of-20
Name: col0, dtype: object

# Function signature

In [7]:
?translate

Signature:
translate(
    translatable: ~Translatable,
    names: Union[~NameType, Iterable[~NameType]] = None,
    ignore_names: Union[~NameType, Iterable[~NameType], Callable[[~NameType], bool]] = None,
    inplace: bool = False,
    override_function: Callable[[~NameType, Set[~SourceType], List[~IdType]], Union[~SourceType, Dict[~SourceType, List[~IdType]], NoneType]] = None,
    maximal_untranslated_fraction: float = 1.0,
    reverse: bool = False,
    attribute: str = None,
) -> Optional[~Translatable]
Docstring:
Translate IDs to human-readable strings.

Args:
    translatable: A data structure to translate.
    names: Explicit names to translate. Derive from `translatable` if ``None``.
    ignore_names: Names **not** to translate, or a predicate ``(str) -> bool``.
    inplace: If ``True``, translate in-place and return ``None``.
    override_function: A callable ``(value, candidates, ids)`` returning one of

        * ``None`` (use regular mapping logic)
        * a ``source`` to